# INTERMEDIATE  MACHINE LEARNING

### Missing Values

A la hora de modelar nuestra data, puede haber casos en los que nos encontremos con valores vacíos y muchas de las librerías (como *scikit-learn*) de Machine Learning darán error a la hora de intentar construir el modelo. Para lidiar con ello, elegiremos algunas de las siguientes estrategias:

* **Eliminar las columnas con los valores perdidos**: Con esta técnica, nos arriesgamos a perder información potencialmente útil en el modelo. Si hay 10k filas, y solo 1 está vacía, perderíamos toda la columna solo por ese valor.

* **Imputación**: Consiste en rellenar los valores vacíos con algún dato aleatorio. El valor generalmente no será el correcto, pero nos dará un modelo más preciso que eliminando toda la información de la columna. 

* **Extensión de imputación**: En este caso, hacemos una imputación, pero también registramos en una nueva columna si los valores originales de la columna estaban vacíos o no. En algunos casos, ayudará a hacer el modelo más preciso, y en otros casos no ayudará en nada. 

Empezaremos viendo un ejemplo con el dataset que tenemos. Predeciremos el precio, y para simplificar, solo usaremos predictores numéricos

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('melb_data.csv')

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

Vamos a crear una función para comparar las diferentes métodos que tenemos al trabajar con valores vacíos. Para ello estudiaremos el **_Mae_** de un modelo Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

Comparemos las diferentes opciones que hemos visto antes para tratar con valores vacíos


**Método 1: Eliminar las columnas**

Tenemos que tener cuidado a la hora de seleccionar la data para hacer el modelo y para testear de que en ambos sets eliminemos las mismas columnas 

In [ ]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


**Método 2: Imputación**

Para la imputación, vamos a usar el método SimpleImputer de la librería *sklearn* para sustituir los valores vacíos con el valor medio de cada columna. 

In [ ]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


Como podemos observar, el método 2 nos ha dado un MAE menor que el método 1, por lo que sería una mejor aproximación.

**Método 3: Extensión de imputación**

Vamos a imputar ahora los valores vacíos mientras registramos que valores han sido imputados.

In [ ]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


Observamos que el MAE de este método es ligeramente peor que el del método 2, por lo que no nos proporciona ninguna mejora significativa registrar que valores han sido cambiados.

**Conclusión**

Hemos podido observar que imputar los valores vacíos es mejor que eliminar las columnas con valores vacíos. Esto se debe a que en este modelo, solo 3 columnas del dataset tienen valores vacíos, y de esas 3 columnas, todas tienen menos de la mitad de filas vacías, por lo que al eliminarlas, perderíamos más de la mitad de la información que nos podría ser significativa. Puede que en otros modelos se dé otro caso donde eliminar las columnas de mejor resultado.

In [21]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = X_train.isnull().sum()

# Filter columns of the serie which value is higher than 0
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64
